## make diffusion samples

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
from matplotlib.image import imread

In [ ]:
import pickle

In [ ]:
base_path= '/home/jinny/projects/Art-history/Art-history/'

file_info = pd.read_csv(base_path+'datas/file_info.csv')
avetor = np.load(base_path+'datas/vectors/avec_latents.npy', allow_pickle=True)

# file_info_latent
df = pd.DataFrame((avetor),columns=['latent','Path']) 
file_info_latents = pd.merge(file_info, df, how = 'left', on = 'Path')
file_info_latents = file_info_latents[~file_info_latents.latent.isnull()]

In [ ]:
with open(base_path+'datas/vectors/diffusion/diffusion_sample.pkl', 'rb') as file:
    painting = pickle.load(file)

## diffusion test

In [ ]:
base_path = '/home/jinny/projects/Art-history/Art-history/datas/vectors/diffusion/no_comma/img/'
temp01 = dict()
temp02 = dict()
for step in ['step5','step10','step20','step30'] :
    with open(base_path + step+'/using_prompt.pkl', 'rb') as file:
        temp01[step] = pickle.load(file)
    with open(base_path + step+'/using_noprompt.pkl', 'rb') as file:
        temp02[step] = pickle.load(file)

In [ ]:
print(len(temp01['step30'][1500]))
temp01['step30'][1500][100]

## make cvec

In [ ]:
import torch
import pickle

In [ ]:
from PIL import Image

In [ ]:
from clip_interrogator import Config, Interrogator

In [ ]:
from tqdm import tqdm

In [ ]:
device = torch.device(f'cuda:{2}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU
print ('Current cuda device ', torch.cuda.current_device()) # check
ci = Interrogator(Config(clip_model_name="ViT-H-14/laion2b_s32b_b79k",device=device))

In [ ]:
base_path = '/home/jinny/projects/Art-history/Art-history/datas/vectors/diffusion/no_comma/cvec/'

for step in tqdm(['step5','step10','step20','step30']) :
    c_result = dict()
    for year in list(temp01[step].keys()) :
        temp = list()
        for i in temp01[step][year] :
            temp.append(ci.image_to_features(i).cpu())
        c_result[year] = temp
    with open(base_path + step+'/using_prompt.pkl', 'wb') as file:
        pickle.dump(c_result, file)
    
    c_result = dict()
    for year in list(temp02[step].keys())  :
        temp = list()
        for i in temp02[step][year] :
            temp.append(ci.image_to_features(i).cpu())
        c_result[year] = temp
    with open(base_path + step+'/using_noprompt.pkl', 'wb') as file:
        pickle.dump(c_result, file)